In [2]:
import open3d as o3d
import numpy as np
import os

# from pyFM.functional import p2p_to_FM
# from pyFM.spectral import LB

import polyscope as ps
ps.init()

import meshplot as mp
def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3


In [3]:
mesh_1 = o3d.io.read_triangle_mesh(os.path.abspath("../simulation/meshes_dancer/frame_0001.ply"))
mesh_2 = o3d.io.read_triangle_mesh(os.path.abspath("../simulation/meshes_dancer/frame_0050.ply"))

mesh_1.compute_vertex_normals()
mesh_2.compute_vertex_normals()

verts_1 = np.asarray(mesh_1.vertices)
verts_2 = np.asarray(mesh_2.vertices)

o3d.visualization.draw_geometries([mesh_1] + [mesh_2])

In [7]:
# KDTree for mesh_2 - target mesh (pointwise goes 2 -> 1 and functional map goes 1 -> 2 if I remeber correctly)
pcd_2 = o3d.geometry.PointCloud() # create pointcloud
pcd_2.points = o3d.utility.Vector3dVector(verts_2) # assign verts with normals to cloud
kdtree = o3d.geometry.KDTreeFlann(pcd_2)

# search for nearest neighbour in mesh_1
p2p_map = []
for vert in verts_1:
    [_, idx, _] = kdtree.search_knn_vector_3d(vert, 1) 
    p2p_map.append(idx[0])

p2p_map = np.array(p2p_map)

In [10]:
faces_1 = np.asarray(mesh_1.triangles)
vertices_1 = np.asarray(mesh_1.vertices)
ps_mesh_1 = ps.register_surface_mesh("mesh_1", vertices_1, faces_1)

faces_2 = np.asarray(mesh_2.triangles)
vertices_2 = np.asarray(mesh_2.vertices)
ps_mesh_2 = ps.register_surface_mesh("mesh_2", vertices_2, faces_2)

# cmaps
cmap_1 = np.linspace(0,1,len(vertices_1)) # gradient for mesh_1
cmap_2 = cmap_1[p2p_map] # map to mesh 2

print(cmap_1)

ps_mesh_1.add_color_quantity("cmap_1", cmap_1, enabled=True)
ps_mesh_2.add_color_quantity("cmap_2", cmap_2, enabled=True)

ps.show()
ps.remove_all_structures() # clear the whole scene

[0.00000000e+00 5.46003527e-06 1.09200705e-05 ... 9.99989080e-01
 9.99994540e-01 1.00000000e+00]


ValueError: 'values' should be an Nx3 array

In [ ]:
k_eigs = 50 # for now, will check what the examples use
eigvects_1, eigvals_1 = LB(mesh_1, k_eigs)
eigvects_2, eigvals_2 = LB(mesh_2, k_eigs)

C_init = p2p_to_FM(p2p_map, eigvects_1, eigvects_2)

cmap1 = visu(mesh_1.vertlist); cmap2 = cmap1[p2p_map]
double_plot(mesh_1, mesh_2, cmap1, cmap2)